
## Import Library


meghubungkan ke google drive

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# preprocessing
import re, string
from spacy.lang.id import Indonesian
import spacy

# Prediksi Klasifikasi
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import joblib

# Deteksi Objek
import os
import cv2
import torch
from PIL import Image

## Dataset

In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/skripsi/teks/X_train_baru_2.csv")

In [4]:
dfJson = pd.read_json("/content/drive/MyDrive/skripsi/test/JSON/Buku Besar Kas/08_Bukubesarkas_Agustus.json")

dfJsonT = dfJson.transpose()

In [5]:
dfJsonT

,text,imagePath
page_1,BADAN PUSAT STATISTIK\nPAPUA BARAT\nBUKU BESAR...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_10,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_11,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_12,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_13,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_14,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_15,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_16,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_2,BADAN PUSAT STATISTIK\nPAPUA BARAT\nBUKU BESAR...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...
page_3,BADAN PUSAT STATISTIK\nPAPUA BARAT\nKdL :\nBUK...,images\Buku Besar Kas\img11\08_Bukubesarkas_Ag...


In [6]:
# menggabungkan beberapa halaman pada dokumen menjadi satu baris dataframe
dfJsonBaru = ''.join(dfJsonT.text)
dt = {'teks' : [dfJsonBaru]}
data = pd.DataFrame(dt)

In [7]:
data

,teks
0,BADAN PUSAT STATISTIK\nPAPUA BARAT\nBUKU BESAR...


In [14]:
# mengecek isi dokumen apakah kosong atau ada isinya untuk keseluruhan dokumen
import sys
if data['teks'][0] == '':
  sys.exit("Dokumen tidak valid. Mohon unggah dokumen yang sesuai")

## Preprocessing Data Input Dokumen

*cleaning text*

In [8]:
def casefolding(text):
  text = text.lower()
  text = text.replace("_<.*?/\|-_:^()>", ' ')
  text = text.strip()
  text = re.compile('<.*?/\|-_:^()>——').sub('', text)
  text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
  text = text.translate(str.maketrans("","",string.punctuation))
  text = re.sub(r"\d+","", text)
  text = re.sub('\+s', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  text = text.replace("\n", ' ')
  text = text.replace("/\-———————— « — —", ' ')
  text = text.encode('ascii', 'ignore').decode('ascii')
  text = re.sub(r'[^\x00-\x7f]', "", text)

  return text

*tokenize*

In [9]:
def tokenisasi(text):
  tokens = text.split(" ")
  return tokens

*stopword removal/stopword elimination*

In [10]:
def stopword_elim(text):
    nlp = Indonesian()
    nlp = spacy.blank('id')
    stopwords = nlp.Defaults.stop_words
    tokens = tokenisasi(text)
    tokens_nostopword = [w for w in tokens if not w in stopwords]
    return (" ").join(tokens_nostopword)

Memanggil function masing-masing preprocessing

In [11]:
def preprocess(string):
  return stopword_elim(casefolding(string))

In [12]:
data['clean_text'] = data['teks'].apply(lambda x: preprocess(x))

In [13]:
data

,teks,clean_text
0,BADAN PUSAT STATISTIK\nPAPUA BARAT\nBUKU BESAR...,badan pusat statistik papua barat buku kas n a...


## Prediksi Klasifikasi

In [16]:
X_train = np.array(df_train['0'].values.astype('U'))

In [17]:
X_baru = np.array(data['clean_text'].values.astype('U'))

*Feature selection* menggunakan pembobotan TF-IDF

In [18]:
tfidf_vectorizer = TfidfVectorizer(use_idf = True, encoding='latin-1').fit(X_train)

X_baru_tfidf = tfidf_vectorizer.transform(X_baru).toarray()

Prediksi dengan model yang sudah ada

In [19]:
teks_path = "/content/drive/MyDrive/skripsi/teks/model_mnb_baru"

In [20]:
def prediksi(path):
  loaded_model = joblib.load(path)
  y_pred_mnb = loaded_model.predict(X_baru_tfidf)
  hasil = y_pred_mnb[0]
  return hasil

In [21]:
pred = prediksi(teks_path)

In [22]:
pred

3

### Output

In [23]:
text = ''
if pred == 3:
  text ="Dokumen Valid, dokumen merupakan Buku Besar Kas"
else:
  text = "Dokumen tidak valid. Mohon unggah dokumen yang sesuai"
print(text)

Dokumen Valid, dokumen merupakan Buku Besar Kas
